In [18]:
import pandas as pd

In [19]:
df1=pd.DataFrame({"id":[1,2],"Nombres":["Juan","Jose"]})
df2=pd.DataFrame({"id":[1,3],"Nombres":["Juan","David"]})

In [20]:
df1

,id,Nombres
0,1,Juan
1,2,Jose


In [21]:
df2

,id,Nombres
0,1,Juan
1,3,David
